In [ ]:
import cv2
import numpy as np
import os
import copy
from matplotlib import pyplot as plt
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from sklearn.cluster import MeanShift
from random import randint
#from detector import Detectors
#from scipy.spatial import distance as dist

In [ ]:
def gray_img(img):
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [ ]:
def read_img(path):
    sequence_tif = {}
    gary_seq_tif = {}
    for root, dirs, files, in os.walk(path):
        dirs.sort()
        for dir in dirs:
            sequence_tif[dir] = []
            gary_seq_tif[dir] = []
            for _, _, files_1 in os.walk(os.path.join(path, dir)):
                files_1.sort()
                for file in files_1:
                    new_dir_path = os.path.join(path, dir)
                    temp_img = cv2.imread(os.path.join(new_dir_path, file))
                    gray_imgs = gray_img(temp_img.copy())
                    sequence_tif[dir].append(temp_img)
                    gary_seq_tif[dir].append(gray_imgs)

    return sequence_tif,gary_seq_tif

In [ ]:
#ass1
def max_filter(img,N,m):
    imgA=np.zeros_like(img)
    #imgA=img.copy()
    padding_img=cv2.copyMakeBorder(img,m,m,m,m,cv2.BORDER_CONSTANT)
    for r in range(img.shape[0]):
        for c in range(img.shape[1]):
            new=padding_img[r:r+N,c:c+N].copy()
            #new=new*mask
            minval,maxval,minindex,maxindex=cv2.minMaxLoc(new)
            imgA[r,c]=maxval
    return imgA
def min_filter(img,N,m):
    imgB=np.zeros_like(img)
    #imgB=img.copy()
    padding_img=cv2.copyMakeBorder(img,m,m,m,m,cv2.BORDER_CONSTANT)
    for r in range(img.shape[0]):
        for c in range(img.shape[1]):
            new=padding_img[r:r+N,c:c+N].copy()
            #new=new*mask
            minval,maxval,minindex,maxindex=cv2.minMaxLoc(new)
            imgB[r,c] = minval
    return imgB
def subtract_img(img1,img2):
    img_O=np.zeros_like(img1)
    for r in range(img1.shape[0]):
        for c in range(img1.shape[1]):
            img_O[r,c]=img1[r,c].astype('int32')-img2[r,c].astype('int32')
            #if img_O[r,c]<=0:
                #img_O[r,c]+=255
        
    min=np.min(img_O)
    max=np.max(img_O)
    for r in range(img_O.shape[0]):
        for c in range(img_O.shape[1]):
            img_O[r,c]=(img_O[r,c]-min)*(255/(max-min))
        
    img_O=img_O.astype(np.uint8)
    return img_O

In [ ]:
def task1(img,gray):
    N=27
    m=N//2
    imgA=min_filter(gray,N,m)
    imgB=max_filter(imgA,N,m)
    imgO=subtract_img(img,imgB)
    imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
    ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Deal with the situation where cells are connected together
    d,contours,hirearchy=cv2.findContours(thresh0, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #Find connected domains
    #comparing the area of connected domains
    contours1=[] 
    for i in contours:
         if cv2.contourArea(i)>25:  #Remove small connected areas
            contours1.append(i)
    #draw circles around cells
    draw=cv2.drawContours(img,contours1,-1,(0,255,0),1)
    # mark the number
    for i,j in zip(contours1,range(len(contours1))):
        M = cv2.moments(i)
        cX=int(M["m10"]/M["m00"])
        cY=int(M["m01"]/M["m00"])
        draw1=cv2.putText(draw, str(j), (cX, cY), 1,1, (255, 0, 255), 1)
    #plt.imshow(draw1,'gray')
    #for index in range(len(contours1)-1):
    #    plt.imsave('cell1_'+str(index)+'.jpg',draw1)
    return divide

In [ ]:
def track_objects(sequence):
        track_types={"BOOSTING":cv2.TrackerBoosting_create, "MIL":cv2.TrackerBoosting_create, "KCF":cv2.TrackerBoosting_create, "TLD":cv2.TrackerBoosting_create, "MEDIANFLOW":cv2.TrackerBoosting_create, "GOTURN":cv2.TrackerBoosting_create, "MOSSE":cv2.TrackerBoosting_create, "CSRT":cv2.TrackerBoosting_create}
        multitracker=track_types['KCF']()
        for i in range(len(sequence)):
            frame=sequence[i]
            g=gray_img(frame)
            orig=copy.copy(frame)
            t=task1(frame,g)
            plt.imsave('task2_0'+str(i)+'.jpg',t)

In [ ]:
path='/Users/admin/Downloads/COMP9517 20T2 Group Project Image Sequences-2/'

In [ ]:
#len(read_img(path)[0]['Sequence 1'])
#read_img(path)[0]['Sequence 1'][i][j]

In [ ]:
path2=path+'PhC-C2DL-PSC'
original_img,grays=read_img(path2)
track_objects(original_img['Sequence 1'])